In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import pickle
from  ModuleNo7My import Numerical_Inversion
from  ModuleNo7My import make_tr_va_te
from  ModuleNo7My import month_range

import ipynb_path
now_file_name = ipynb_path.get().split('/')[-1].replace('.ipynb','')

input_path = '../input/'
status_file_name = 'status.csv'
station_file_name = 'station.csv'
trip_file_name = 'trip.csv'
weather_file_name = 'weather.csv'
trip_start_date_file_name = 'trip_start_date.csv'
trip_end_date_file_name = 'trip_end_date.csv'
train_sta_wea_tri_end_file_name = 'train_all_row.csv'
train_sta_wea_notna_all_file_name = 'all_row_notna.csv'

output_path = '../output/'
model_path = '../model/'

In [2]:
# #実行したらコメントアウトする
# #保存ファイルの重複が起こる可能性がある
# ##########################################################
# # もう一度notebookを動かす際はモデルリストのstart_numとend_numを
# # 設定してください
# ##########################################################
# st_in = input('初めの数字を入力してください')
# end_in = input('終わりの数字を入力してください')
# start_num = int(st_in)
# end_num = int(end_in)

# model_name_list = [f'model_{i}_{now_file_name}.sav' for i in range(start_num, end_num)]
# submit_file_name_list = [f'submission_{i}_{now_file_name}.csv' for i in range(start_num,end_num)]
# model_submit_dict = dict(zip(model_name_list,submit_file_name_list))

初めの数字を入力してください 0
終わりの数字を入力してください 20


In [3]:
status = pd.read_csv(input_path + status_file_name)
station = pd.read_csv(input_path + station_file_name)
weather = pd.read_csv(input_path + weather_file_name)

In [7]:

# #statusのyear, month, dayを結合してdatetime型に
# status['date'] = status['year'].astype(str) + '/' + status['month'].astype(str).str.zfill(2).astype(str) + '/' + status['day'].astype(str).str.zfill(2).astype(str)
# status['date'] = pd.to_datetime(status['date'])

# weather['date'] = pd.to_datetime(weather['date'])

# train_sta_wea = pd.merge(status,weather, on = 'date', how ='left')

# train_sta_wea['events'] = train_sta_wea['events'].fillna('なし')

# le = preprocessing.LabelEncoder()
# train_sta_wea['events'] = le.fit_transform(train_sta_wea['events'].values.tolist())
# train_sta_wea = pd.merge(train_sta_wea, station[['station_id', 'city']], how = 'left')

# #曜日を追加
# train_sta_wea['date']=pd.to_datetime(train_sta_wea['date'])
# train_sta_wea['weekday']=train_sta_wea['date'].dt.weekday
# train_sta_wea.head()



# X=train_sta_wea.drop(['id','predict','bikes_available','date'],axis=1)
# X_city=pd.get_dummies(X.loc[:,'city'],prefix='city')
# X=pd.concat([X,X_city],axis=1)
# X=X.drop('city',axis=1)

# X_all = pd.concat([X, train_sta_wea[['id','predict','bikes_available','date']]], axis=1)

In [8]:
# #basic_allとして保存
# train_sta_wea.to_csv(output_path + 'basic_all.csv',index=False)

In [11]:
# #X_allとして保存
# X_all.to_csv(input_path + 'X_all.csv',index=False)

In [ ]:
X_all = pd.read_csv(input_path + 'X_all.csv', index=False)

In [12]:
param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbose_eval': 10000,
        'boosting_type': 'gbdt',
    }

In [17]:
#optuna.integration.LightGBMを使用する
import optuna.integration.lightgbm as lgb
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import time

main_df = X_all.copy()
valid_score_list = []

ts = time.time()

count = 1
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(main_df, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    
    te_X = make_data.predict_for_data(test)
    lgb_train = lgb.Dataset(tr_X, tr_y)
    lgb_eval = lgb.Dataset(va_X, va_y)
    
    booster = lgb.train(param, 
                 lgb_train,
                 valid_sets=lgb_eval,
                 verbose_eval=10000,
                 early_stopping_rounds=100)
    
    best_score = booster.best_score['valid_0']['rmse']
    valid_score_list.append(best_score)
                       
    print(f"Best Params Fold {count}:", booster.params)
    print(f"Best Iteration Fold {count}:", booster.best_iteration)
    print(f"Best Score(rmse) Fold {count}:", best_score)
    print('time is :', time.time() - ts)

    count += 1
    
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print('time is :', time.time() - ts)

[I 2021-10-21 10:36:47,317] A new study created in memory with name: no-name-a523e6ad-884e-4a05-aa4d-e7149b2bfc48
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017401 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 942
[LightGBM] [Info] Number of data points in the train set: 539515, number of used features: 32
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Start training from score 8.601260
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 3.211219:  14%|#4        | 1/7 [00:14<01:26, 14.48s/it]

Early stopping, best iteration is:
[524]	valid_0's rmse: 3.21122
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022044 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 942
[LightGBM] [Info] Number of data points in the train set: 539515, number of used features: 32
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Start training from score 8.601260
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 3.185993:  29%|##8       | 2/7 [00:23<01:04, 12.97s/it]

...

Early stopping, best iteration is:
[93]	valid_0's rmse: 3.34675
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1047
[LightGBM] [Info] Number of data points in the train set: 771635, number of used features: 32
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Start training from score 8.495921
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 3.346746:  80%|########  | 4/5 [00:45<00:12, 12.39s/it]

Early stopping, best iteration is:
[235]	valid_0's rmse: 3.35765
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029527 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1047
[LightGBM] [Info] Number of data points in the train set: 771635, number of used features: 32
[LightGBM] [Warning] Unknown parameter: verbose_eval
[LightGBM] [Info] Start training from score 8.495921
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 3.346746: 100%|##########| 5/5 [00:54<00:00, 10.90s/it]

Early stopping, best iteration is:
[42]	valid_0's rmse: 3.37259
Best Params Fold 12: {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.00036428356208070877, 'lambda_l2': 0.0039939935794538126, 'num_leaves': 81, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 100, 'num_iterations': 1000, 'early_stopping_round': 100}
Best Iteration Fold 12: 93
Best Score(rmse) Fold 12: 3.3467464659157673
time is : 11278.730734109879
***
***
CV score is 3.4241157719453614
time is : 11278.785489320755


In [19]:
fold1_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 1.9696397630915266e-07, 'lambda_l2': 1.3661669869987752e-05, 'num_leaves': 135, 'feature_fraction': 0.716, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold2_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.011146367340810535, 'lambda_l2': 0.0011998022565447625, 'num_leaves': 156, 'feature_fraction': 1.0, 'bagging_fraction': 0.6277266009025795, 'bagging_freq': 2, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold3_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.0005967431045214434, 'lambda_l2': 0.003922249609177241, 'num_leaves': 120, 'feature_fraction': 0.8999999999999999, 'bagging_fraction': 0.4467288764999327, 'bagging_freq': 1, 'min_child_samples': 50, 'num_iterations': 1000, 'early_stopping_round': 100}
fold4_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.007609178302073965, 'lambda_l2': 1.4222704070110468e-08, 'num_leaves': 92, 'feature_fraction': 0.6, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold5_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.0006351587224033728, 'lambda_l2': 1.0423294917197932e-08, 'num_leaves': 42, 'feature_fraction': 0.8, 'bagging_fraction': 0.5302742170315369, 'bagging_freq': 5, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold6_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 2.2873303643975347e-07, 'lambda_l2': 2.5313632221822466e-06, 'num_leaves': 256, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold7_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 6.3945592896534365, 'lambda_l2': 0.020553505585021524, 'num_leaves': 239, 'feature_fraction': 1.0, 'bagging_fraction': 0.8810461837655519, 'bagging_freq': 3, 'min_child_samples': 10, 'num_iterations': 1000, 'early_stopping_round': 100}
fold8_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.05022428549717312, 'lambda_l2': 0.0002165271229855614, 'num_leaves': 169, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold9_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 4.3410881360206773e-07, 'lambda_l2': 2.2526764761862363, 'num_leaves': 256, 'feature_fraction': 1.0, 'bagging_fraction': 0.6629800362718559, 'bagging_freq': 6, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold10_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 2.111922443412253, 'lambda_l2': 0.7235544829049594, 'num_leaves': 174, 'feature_fraction': 0.7, 'bagging_fraction': 0.7993541638554258, 'bagging_freq': 2, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold11_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.07619056989177418, 'lambda_l2': 9.957713402710954e-06, 'num_leaves': 184, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
fold12_params = {'objective': 'regression', 'metric': 'rmse', 'verbose_eval': 10000, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.00036428356208070877, 'lambda_l2': 0.0039939935794538126, 'num_leaves': 81, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 100, 'num_iterations': 1000, 'early_stopping_round': 100}



In [49]:
for i in range(1, 13):
    exec(f'tmp_params = fold{i}_params')
    tmp_dict =  {key : [value] for key, value in tmp_params.items()}
    exec(f'fold{i}_pre_dict = tmp_dict')
    
    exec(f'fold{i}_pre_dict')

In [50]:
for i in range(1,13):
    exec(f'df_fold{i} = pd.DataFrame(fold{i}_pre_dict)')

In [51]:
all_fold = pd.DataFrame(columns=df_fold1.columns)
for i in range(1, 13):
    print(i)
    exec(f'all_fold = pd.concat([all_fold, df_fold{i}])')


1
2
3
4
5
6
7
8
9
10
11
12


In [52]:
all_fold

,objective,metric,verbose_eval,boosting_type,feature_pre_filter,lambda_l1,lambda_l2,num_leaves,feature_fraction,bagging_fraction,bagging_freq,min_child_samples,num_iterations,early_stopping_round
0,regression,rmse,10000,gbdt,False,1.969640e-07,1.366167e-05,135,0.716,1.000000,0,20,1000,100
0,regression,rmse,10000,gbdt,False,1.114637e-02,1.199802e-03,156,1.000,0.627727,2,20,1000,100
0,regression,rmse,10000,gbdt,False,5.967431e-04,3.922250e-03,120,0.900,0.446729,1,50,1000,100
0,regression,rmse,10000,gbdt,False,7.609178e-03,1.422270e-08,92,0.600,1.000000,0,20,1000,100
0,regression,rmse,10000,gbdt,False,6.351587e-04,1.042329e-08,42,0.800,0.530274,5,20,1000,100
0,regression,rmse,10000,gbdt,False,2.287330e-07,2.531363e-06,256,1.000,1.000000,0,20,1000,100
0,regression,rmse,10000,gbdt,False,6.394559e+00,2.055351e-02,239,1.000,0.881046,3,10,1000,100
0,regression,rmse,10000,gbdt,False,5.022429e-02,2.165271e-04,169,1.000,1.000000,0,20,1000,100
0,regression,rmse,10000,gbdt,False,4.341088e-07,2.252676e+00,256,1.000,0.662980,6,20,1000,100
0,regression,rmse,10000,gbdt,False,2.111922e+00,7.235545e-01,174,0.700,0.799354,2,20,1000,100


In [53]:
cv_list_tune = [3.1795526962070375, 3.326195014110485, 3.6769541339265372, 3.3865252905423557, 3.6129541698965393, 3.441277125336344, 3.4122856010289015, 3.5355423968775135, 3.307722731448967, 3.4016134815406183, 3.4620201565132755, 3.3467464659157673]

In [54]:
all_fold['best_score'] = cv_list_tune

In [56]:
all_fold.reset_index(drop=True,inplace=True)

In [57]:
all_fold

,objective,metric,verbose_eval,boosting_type,feature_pre_filter,lambda_l1,lambda_l2,num_leaves,feature_fraction,bagging_fraction,bagging_freq,min_child_samples,num_iterations,early_stopping_round,best_score
0,regression,rmse,10000,gbdt,False,1.969640e-07,1.366167e-05,135,0.716,1.000000,0,20,1000,100,3.179553
1,regression,rmse,10000,gbdt,False,1.114637e-02,1.199802e-03,156,1.000,0.627727,2,20,1000,100,3.326195
2,regression,rmse,10000,gbdt,False,5.967431e-04,3.922250e-03,120,0.900,0.446729,1,50,1000,100,3.676954
3,regression,rmse,10000,gbdt,False,7.609178e-03,1.422270e-08,92,0.600,1.000000,0,20,1000,100,3.386525
4,regression,rmse,10000,gbdt,False,6.351587e-04,1.042329e-08,42,0.800,0.530274,5,20,1000,100,3.612954
5,regression,rmse,10000,gbdt,False,2.287330e-07,2.531363e-06,256,1.000,1.000000,0,20,1000,100,3.441277
6,regression,rmse,10000,gbdt,False,6.394559e+00,2.055351e-02,239,1.000,0.881046,3,10,1000,100,3.412286
7,regression,rmse,10000,gbdt,False,5.022429e-02,2.165271e-04,169,1.000,1.000000,0,20,1000,100,3.535542
8,regression,rmse,10000,gbdt,False,4.341088e-07,2.252676e+00,256,1.000,0.662980,6,20,1000,100,3.307723
9,regression,rmse,10000,gbdt,False,2.111922e+00,7.235545e-01,174,0.700,0.799354,2,20,1000,100,3.401613


In [ ]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []

#分析のメインになるdfを設定する
main_df = train_sta_wea.copy()

#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(main_df, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model_path, model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

In [ ]:
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')

In [ ]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

In [ ]:
# ! signate submit --competition-id=550 '/Users/ken/OneDrive - Yokohama City University/活動/data-sci/データ分析遊び/competition/SIGNATE/オペレーション最適化に向けたシェアサイクルの利用予測/output/submission_5_No_7_my.csv'


